In [12]:
#Import Python Libraries

%matplotlib inline
import matplotlib.pyplot as plt
import xml.etree.ElementTree as etree 
from datetime import datetime, timedelta
import os
import json
import pandas as pd
import numpy as np
from pytz import common_timezones, all_timezones, timezone

In [ ]:
def get_file_path(file_name):
    input_dir = os.path.join("/", "opt","jupyter","data","csv" )
    input_file_path = os.path.join(input_dir , file_name)
    return input_file_path

#from admin_garmin import get_file_path
input_file = 'activity_898238015_30.csv'
output_file = 'activity_898238015_40.csv'


input_file_path = get_file_path(input_file)
output_file_path = get_file_path(output_file)


In [21]:
#activity_file = "D:/downloads/activity_873783146.tcx" # Naul cycle with Barry
#activity_file = "D:/downloads/activity_898238015.tcx" # Great Dublin Cycle

# Define name and path of data file
# This data file contains a sub-set of activiy information relating to speed and time

input_file = 'activity_898238015.tcx'  # Great Dublin Cycle
input_dir = os.path.join("/", "opt","jupyter","data","csv" )
input_file_path = os.path.join(input_dir , input_file)

output_file = 'activity_898238015.json'
output_dir = os.path.join("/", "opt","jupyter","data","csv" )
output_file_path = os.path.join(input_dir , output_file)

print ("Input file is (full path)  ", input_file_path)
print ("Output file is (full path)  ", output_file_path)


Input file is (full path)   /opt/jupyter/data/csv/activity_898238015.tcx
Output file is (full path)   /opt/jupyter/data/csv/activity_898238015.json


In [14]:
import xml.etree.ElementTree as etree 
tree = etree.parse( input_file_path ) 
root = tree.getroot()[0][0]
print("Print representation of root: " + str(root) )
print("Print root tag: " + str(root.tag))
print("Print root attribute: " + str(root.attrib))
print("Number of children for root is: " + str(len(root)) )
number_of_laps = len(root) - 2
print("Number of laps is: " + str( number_of_laps ) )

Print representation of root: <Element '{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}Activity' at 0x7fe9436deb38>
Print root tag: {http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}Activity
Print root attribute: {'Sport': 'Biking'}
Number of children for root is: 3
Number of laps is: 1


In [15]:
for i in root:
    print (i)

<Element '{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}Id' at 0x7fe9436de7c8>
<Element '{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}Lap' at 0x7fe9436dec78>
<Element '{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}Creator' at 0x7fe9425549a8>


In [16]:
# Create a List object called my_data

my_data = []

#for lap in [1,2,3]:
for lap in range(1, number_of_laps + 1):
    data = {}
    for i in root[lap][9]:
        #print (i)
        if len(data) > 0:
            my_data.append(data)
        data = {}
        for j in i:
            
            #print(j.tag + ' ' + j.text)
            if "Time" in j.tag:
                #print ("Time is " + j.text)
                #data['time']= j.text
                data['time'] = datetime.strptime(  j.text[0:19], "%Y-%m-%dT%H:%M:%S" )
            elif "DistanceMeters" in j.tag:
                #print ("Distance is " + j.text)
                data['distance']= float(j.text)
            elif "HeartRateBpm" in j.tag:
                for k in j:
                    #print ("Heart Rate is " +  k.text)
                    data['heart_rate']= int(k.text)
            elif "Cadence" in j.tag:
                #print ("Cadence is " + j.text)
                data['cadence']= int(j.text)
            elif "Altitude" in j.tag:
                #print ("Cadence is " + j.text)
                data['altitude']= float(j.text)
            elif "Position" in j.tag:
                for k in j:
                    #print ("Heart Rate is " +  k.text)
                    if "Latitude" in k.tag:
                        data['latitude']= float(k.text)
                    elif "Longitude" in k.tag:
                        data['longitude']= float(k.text)
            else:
                print(j.tag, j.text)
    if len(data) > 0:
        my_data.append(data)
        #print data

In [19]:
my_data[:3]

[{'altitude': 79.4000015258789,
  'cadence': 76,
  'distance': 5.010000228881836,
  'heart_rate': 103,
  'latitude': 53.36370773613453,
  'longitude': -6.190182846039534,
  'time': datetime.datetime(2015, 9, 13, 6, 25, 51)},
 {'altitude': 90.80000305175781,
  'cadence': 83,
  'distance': 28.09000015258789,
  'heart_rate': 106,
  'latitude': 53.363760793581605,
  'longitude': -6.190518541261554,
  'time': datetime.datetime(2015, 9, 13, 6, 25, 56)},
 {'altitude': 90.80000305175781,
  'cadence': 84,
  'distance': 34.7400016784668,
  'heart_rate': 106,
  'latitude': 53.36377898231149,
  'longitude': -6.1906137596815825,
  'time': datetime.datetime(2015, 9, 13, 6, 25, 57)}]

In [20]:
my_data_dict = { 'data_points': my_data }

In [27]:
date_handler = lambda obj: obj.isoformat() if isinstance(obj, datetime) else None
import json
with open(output_file_path, 'w') as outfile:
    json.dump(my_data_dict, outfile,  default = date_handler)

In [30]:
with open(output_file_path, 'r') as json_data:
    d = json.load(json_data)
    

In [39]:
df2 = pd.DataFrame.from_dict(d['data_points'])

In [40]:
df2[:3]

,altitude,cadence,distance,heart_rate,latitude,longitude,time
0,79.400002,76,5.010000,103,53.363708,-6.190183,2015-09-13T06:25:51
1,90.800003,83,28.090000,106,53.363761,-6.190519,2015-09-13T06:25:56
2,90.800003,84,34.740002,106,53.363779,-6.190614,2015-09-13T06:25:57


In [41]:
df2.set_index(['time'], inplace=True)

In [42]:
df2[:3]

,altitude,cadence,distance,heart_rate,latitude,longitude
time,,,,,,
2015-09-13T06:25:51,79.400002,76,5.010000,103,53.363708,-6.190183
2015-09-13T06:25:56,90.800003,83,28.090000,106,53.363761,-6.190519
2015-09-13T06:25:57,90.800003,84,34.740002,106,53.363779,-6.190614


In [46]:
df2.index = pd.to_datetime(df2.index)

In [47]:
df2[:3]

,altitude,cadence,distance,heart_rate,latitude,longitude
2015-09-13 06:25:51,79.400002,76,5.010000,103,53.363708,-6.190183
2015-09-13 06:25:56,90.800003,83,28.090000,106,53.363761,-6.190519
2015-09-13 06:25:57,90.800003,84,34.740002,106,53.363779,-6.190614


In [49]:
df2.index

<class 'pandas.tseries.index.DatetimeIndex'>
[2015-09-13 06:25:51, ..., 2015-09-13 11:32:02]
Length: 5358, Freq: None, Timezone: None

In [36]:
#for i in d['data_points']:
#    print(i['time'])

In [11]:
print(my_data[:3])
my_data_dict = { 'data_points': my_data }
#my_data_json[:300]
#xxxx = json.dumps(my_data_json)
my_data_dict['data_points'][0]['time']
len(my_data_dict['data_points'][:2])
my_data_dict['data_points'][:2]

[{'cadence': 76, 'latitude': 53.36370773613453, 'distance': 5.010000228881836, 'heart_rate': 103, 'time': datetime.datetime(2015, 9, 13, 6, 25, 51), 'longitude': -6.190182846039534, 'altitude': 79.4000015258789}, {'cadence': 83, 'latitude': 53.363760793581605, 'distance': 28.09000015258789, 'heart_rate': 106, 'time': datetime.datetime(2015, 9, 13, 6, 25, 56), 'longitude': -6.190518541261554, 'altitude': 90.80000305175781}, {'cadence': 84, 'latitude': 53.36377898231149, 'distance': 34.7400016784668, 'heart_rate': 106, 'time': datetime.datetime(2015, 9, 13, 6, 25, 57), 'longitude': -6.1906137596815825, 'altitude': 90.80000305175781}]


[{'altitude': 79.4000015258789,
  'cadence': 76,
  'distance': 5.010000228881836,
  'heart_rate': 103,
  'latitude': 53.36370773613453,
  'longitude': -6.190182846039534,
  'time': datetime.datetime(2015, 9, 13, 6, 25, 51)},
 {'altitude': 90.80000305175781,
  'cadence': 83,
  'distance': 28.09000015258789,
  'heart_rate': 106,
  'latitude': 53.363760793581605,
  'longitude': -6.190518541261554,
  'time': datetime.datetime(2015, 9, 13, 6, 25, 56)}]